In [1]:
from langchain.llms import OpenAI
import os
import openai

from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain

from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA

In [3]:

# openai.api_key = os.environ["OPENAI_API_KEY"]

In [7]:
# Before executing the following code, make sure to have
# your OpenAI key saved in the “OPENAI_API_KEY” environment variable.
llm = OpenAI(model="text-davinci-003", temperature=0.9)

In [3]:
text = "Suggest a personalized workout routine for someone looking to improve cardiovascular endurance and prefers outdoor activities."
print(llm(text))



1. Start by taking a walk for 30 minutes at a moderate pace. This should be done at least 3-4 times per week. 

2. After building a base, add some jogging to the routine. Start with an interval of jogging for 1 minute followed by 2 minutes of walking and repeat for 15 minutes. Do this 2-3 times per week.

3. Once comfortable with the jogging, add running intervals. Start with an interval of running for 2 minutes followed by 2 minutes of walking and repeat for 15 minutes. Do this 2-3 times per week.

4. As the body becomes accustomed to the running intervals, add longer running intervals. Start with an interval of running for 5 minutes followed by 2 minutes of walking and repeat for 20 minutes. Do this 2-3 times per week.

5. Supplement the running routine with some strength training. This will help build muscle endurance and increase the overall health and strength of the body. Aim to do strength training 2-3 times per week.

6. Finally, add some outdoor activities to the routine suc

### Chains

In [9]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain

llm = OpenAI(model="text-davinci-003", temperature=0.9)
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

chain = LLMChain(llm=llm, prompt=prompt)

# Run the chain only specifying the input variable.
print(chain.run("eco-friendly water bottles"))



EcoPure Hydration.


### Conversations

In [ ]:
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

llm = OpenAI(model="text-davinci-003", temperature=0)
conversation = ConversationChain(
    llm=llm,
    verbose=True,
    memory=ConversationBufferMemory()
)

# Start the conversation
conversation.predict(input="Tell me about yourself.")

# Continue the conversation
conversation.predict(input="What can you do?")
conversation.predict(input="How can you help me with data analysis?")

# Display the conversation
print(conversation)

### DeepLake

In [13]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA

# Before executing the following code, make sure to have your
# Activeloop key saved in the “ACTIVELOOP_TOKEN” environment variable.

# instantiate the LLM and embeddings models
llm = OpenAI(model="text-davinci-003", temperature=0)
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# create our documents
texts = [
    "Napoleon Bonaparte was born in 15 August 1769",
    "Louis XIV was born in 5 September 1638"
]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.create_documents(texts)

In [14]:
# create Deep Lake dataset
# TODO: use your organization id here. (by default, org id is your username)
my_activeloop_org_id = "maitemarzoratti" 
my_activeloop_dataset_name = "langchain_course_from_zero_to_hero"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"
db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)

LockedException: This dataset cannot be open for writing as it is locked by another machine. Try loading the dataset with `read_only=True`.

In [11]:

# add documents to our Deep Lake dataset
db.add_documents(docs)

NameError: name 'db' is not defined

In [7]:
retrieval_qa = RetrievalQA.from_chain_type(
	llm=llm,
	chain_type="stuff",
	retriever=db.as_retriever()
)

In [8]:
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

tools = [
    Tool(
        name="Retrieval QA System",
        func=retrieval_qa.run,
        description="Useful for answering questions."
    ),
]

agent = initialize_agent(
	tools,
	llm,
	agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
	verbose=True
)

In [9]:
response = agent.run("When was Napoleone born?")
print(response)



> Entering new  chain...
 I need to find out when Napoleone was born.
Action: Retrieval QA System
Action Input: When was Napoleone born?
Observation:  Napoleon Bonaparte was born on 15 August 1769.
Thought: I now know the final answer.
Final Answer: Napoleon Bonaparte was born on 15 August 1769.

> Finished chain.
Napoleon Bonaparte was born on 15 August 1769.


In [10]:
# load the existing Deep Lake dataset and specify the embedding function
db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)

# create new documents
texts = [
    "Lady Gaga was born in 28 March 1986",
    "Michael Jeffrey Jordan was born in 17 February 1963"
]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.create_documents(texts)

# add documents to our Deep Lake dataset
db.add_documents(docs)

Deep Lake Dataset in hub://maitemarzoratti/langchain_course_from_zero_to_hero already exists, loading from the storage


\

Dataset(path='hub://maitemarzoratti/langchain_course_from_zero_to_hero', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype      shape     dtype  compression
  -------    -------    -------   -------  ------- 
 embedding  embedding  (4, 1536)  float32   None   
    id        text      (4, 1)      str     None   
 metadata     json      (4, 1)      str     None   
   text       text      (4, 1)      str     None   


['9174e608-1399-11ee-91aa-d0a637ea8e5b',
 '9174e702-1399-11ee-91aa-d0a637ea8e5b']

In [16]:
db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)

# instantiate the wrapper class for GPT3
llm = OpenAI(model="text-davinci-003", temperature=0)

# create a retriever from the db
retrieval_qa = RetrievalQA.from_chain_type(
	llm=llm, chain_type="stuff", retriever=db.as_retriever()
)

# instantiate a tool that uses the retriever
tools = [
    Tool(
        name="Retrieval QA System",
        func=retrieval_qa.run,
        description="Useful for answering questions."
    ),
]

# create an agent that uses the tool
agent = initialize_agent(
	tools,
	llm,
	agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
	verbose=True
)

Deep Lake Dataset in hub://maitemarzoratti/langchain_course_from_zero_to_hero already exists, loading from the storage


In [17]:
response = agent.run("When was Michael Jordan born?")
print(response)



> Entering new  chain...
 I need to find out when Michael Jordan was born.
Action: Retrieval QA System
Action Input: When was Michael Jordan born?
Observation:  Michael Jordan was born on 17 February 1963.
Thought: I now know the final answer.
Final Answer: Michael Jordan was born on 17 February 1963.

> Finished chain.
Michael Jordan was born on 17 February 1963.


### Google Search

In [2]:
from langchain.llms import OpenAI

from langchain.agents import AgentType
from langchain.agents import load_tools
from langchain.agents import initialize_agent

from langchain.agents import Tool
from langchain.utilities import GoogleSearchAPIWrapper

In [3]:
llm = OpenAI(model="text-davinci-003", temperature=0)

In [4]:
# remember to set the environment variables
# “GOOGLE_API_KEY” and “GOOGLE_CSE_ID” to be able to use
# Google Search via API.
search = GoogleSearchAPIWrapper()

tools = [
    Tool(
        name = "google-search",
        func=search.run,
        description="useful for when you need to search google to answer questions about current events"
    )
]

agent = initialize_agent(tools, 
                         llm, 
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
                         verbose=True,
                         max_iterations=6)

In [12]:
tools[0].run("Obama's first name?")

"STATE OF HAWAII. 1 Child's First Name. (Type or print). BARACK. CERTIFICATE OF LIVE BIRTH lb. Middle Name. HUSSE. HUSSEIN. FILE. NUMBER 151. Barack Hussein Obama II is an American politician who served as the 44th president of the United States from 2009 to 2017. A member of the Democratic Party,\xa0... His full name is Barack Hussein Obama II. Since the “II” is simply because he was named for his father, his last name is Obama. Jan 19, 2017 ... Jordan Barack Treasure, New York City, born in 2008 ... Jordan Barack Treasure made national news when he was the focus of a New York newspaper\xa0... Portrait of George Washington, the 1st President of the United States ... Portrait of Barack Obama, the 44th President of the United States\xa0... Feb 9, 2015 ... Michael Jordan misspelled Barack Obama's first name on 50th-birthday gift ... and he said, 'I can't put this up, he misspelled my name! First Lady Michelle LaVaughn Robinson Obama is a lawyer, writer, and the wife of the 44th President

In [5]:
response = agent("What's the latest news about the Mars rover?")
print(response['output'])



> Entering new  chain...
 I need to find out the latest news about the Mars rover
Action: google-search
Action Input: "latest news Mars rover"
Observation: The mission has concluded that the solar-powered lander has run out of energy after more than four years on the Red Planet. Dec 15, 2021 ... Mars Sample Return is going to have great stuff to choose from!” Get the Latest JPL News. SUBSCRIBE TO THE NEWSLETTER. NASA's Curiosity rover is discovering odd rocks halfway into a region called the "clay-bearing unit." ... This panorama of a location called "Teal Ridge" was ... Curiosity rover finds water-carved 'book' rock on Mars (photo) ... A book-shaped rock on Mars was likely carved by ancient water running through the area, ... Get the latest news releases about the Perseverance rover mission. Apr 25, 2023 ... China's Mars rover, which has been in longer-than-expected hibernation on the red planet since May 2022, has likely suffered excessive ... Apr 28, 2023 ... Friday's news comes d

In [6]:
llm = OpenAI(model="text-davinci-003", temperature=0)

prompt = PromptTemplate(
    input_variables=["query"],
    template="Write a summary of the following text: {query}"
)

summarize_chain = LLMChain(llm=llm, prompt=prompt)

In [7]:
# remember to set the environment variables
# “GOOGLE_API_KEY” and “GOOGLE_CSE_ID” to be able to use
# Google Search via API.
search = GoogleSearchAPIWrapper()

tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for finding information about recent events"
    ),
    Tool(
       name='Summarizer',
       func=summarize_chain.run,
       description='useful for summarizing texts'
    )
]

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True  
)

In [12]:
response = agent("What are the latest news on the elections in argentina? Then summarize the results.")
print(response['output'])



> Entering new  chain...
 I need to find the latest news and then summarize it.
Action: Search
Action Input: Latest news on the elections in Argentina
Observation: Oct 28, 2019 ... Centre-left opposition candidate Alberto Fernández has been elected president of Argentina in a vote dominated by economic concerns. May 14, 2023 ... Argentina's ruling Peronist party wins provincial elections ... May 14 - Argentina's ruling Peronist party posted strong wins in elections Sunday, ... May 5, 2023 ... Polls have him capturing as much as a quarter of potential votes. The current leftist president says he won't run for reelection, and the right ... Jun 1, 2023 ... A new Executive Council was also elected. “In these times when inequality and climate change are the greatest global threats, the WMO must ... Apr 21, 2023 ... Argentine President Alberto Fernandez will not run in the country's ... German Chancellor Olaf Scholz (not pictured) hold a news conference, ... May 4, 2023 ... In October Arge